In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Main data and New data

In [16]:
df = pd.read_csv(r'C:\Users\Minfy.DESKTOP-3E50D5N\Documents\mlflow_ml\Bank_Personal_Loan_Modelling.csv')
df.shape

(5000, 14)

In [17]:
df_test = pd.read_csv(r'c:\Users\Minfy.DESKTOP-3E50D5N\Downloads\New Customer Bank_Personal_Loan.csv')
df_test.shape

(15, 13)

In [18]:
df_main = df.drop(columns=['Personal Loan'])

In [19]:
df_main.columns

Index(['ID', 'Age', 'Experience', 'Income', 'ZIP Code', 'Family', 'CCAvg',
       'Education', 'Mortgage', 'Securities Account', 'CD Account', 'Online',
       'CreditCard'],
      dtype='object')

In [20]:
df_main.shape, df_test.shape

((5000, 13), (15, 13))

In [21]:
import pandas as pd
from evidently import Report
from evidently.presets import DataDriftPreset

In [22]:

reports = Report([DataDriftPreset(columns=[ 'Age', 'Experience', 'Income', 'Family', 'CCAvg',
       'Education', 'Mortgage', 'Securities Account', 'CD Account', 'Online',
       'CreditCard'])])
report=reports.run(reference_data=df_main, current_data=df_test)


In [23]:
report.save_html("evidently_Report.html")

In [11]:
drift_metrix = report.json()
drift_metrix

'{"metrics": [{"id": "45850336a9bbeb8c466e31a93cc786ce", "metric_id": "DriftedColumnsCount(columns=Age,Experience,Income,Family,CCAvg,Education,Mortgage,Securities Account,CD Account,Online,CreditCard,drift_share=0.5)", "value": {"count": 10.0, "share": 0.9090909090909091}}, {"id": "8f5d1c60a32d6fc1bd54bc53af61d8e8", "metric_id": "ValueDrift(column=Age)", "value": 0.6921177228051255}, {"id": "d7a4a7f023b351f2a15c5725bddd463d", "metric_id": "ValueDrift(column=Experience)", "value": 0.6715906280833273}, {"id": "c0d4e4e971dc7208709fd8de755d0ee9", "metric_id": "ValueDrift(column=Income)", "value": 0.2398756546956939}, {"id": "514b7f0c04de7eb83ad9016cb4863e07", "metric_id": "ValueDrift(column=Family)", "value": 0.16127758354467472}, {"id": "1cdb62269d4036059a9bb0bb92c37a2d", "metric_id": "ValueDrift(column=CCAvg)", "value": 0.19465125861911411}, {"id": "12a293d8c8fad4b7911d782a75e321d1", "metric_id": "ValueDrift(column=Education)", "value": 0.1222959142354372}, {"id": "769ab51f15516ec5c50b6

In [12]:
import json
import mlflow
import re

# Set the MLflow tracking URI to your server's URL
mlflow.set_tracking_uri("http://localhost:5000")  # change to your MLflow tracking server address
mlflow.set_experiment("Evidently Drift Metrics")  # change to your experiment name
json_str = drift_metrix
report_dict = json.loads(json_str)

def clean_metric_name(name):
    if "ValueDrift(column=" in name:
        match = re.search(r"ValueDrift\(column=(.*?)\)", name)
        if match:
            return f"drift_{match.group(1)}"
    elif "DriftedColumnsCount" in name:
        return "drifted_column_count"
    cleaned = re.sub(r"[^a-zA-Z0-9_\-./]", "_", name)
    return cleaned[:50]

with mlflow.start_run() as run:
    for metric in report_dict["metrics"]:
        raw_name = metric["metric_id"]
        value = metric["value"]
        if isinstance(value, dict):
            for k, v in value.items():
                metric_name = f"{raw_name}_{k}"
                cleaned_name = clean_metric_name(metric_name)
                mlflow.log_metric(cleaned_name, v)
        else:
            cleaned_name = clean_metric_name(raw_name)
            mlflow.log_metric(cleaned_name, value)

    with open("evidently_report.json", "w") as f:
        json.dump(report_dict, f, indent=2)
    mlflow.log_artifact("evidently_report.json")


🏃 View run sincere-tern-555 at: http://localhost:5000/#/experiments/415887779351301810/runs/f5174bfeeed442c199105084ade779d4
🧪 View experiment at: http://localhost:5000/#/experiments/415887779351301810


# test and train split data drift

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(df_main, test_size=0.2, random_state=42)

In [25]:
X_train.shape, X_test.shape

((4000, 13), (1000, 13))

In [29]:
report1 = Report([DataDriftPreset()])

report1 = report1.run(reference_data=X_train, current_data=X_test)   

In [ ]:
report1_metrix = report1.json()
report1.save_html("evidently_Report1.html")

In [32]:
import json
import mlflow
import re

# Set the MLflow tracking URI to your server's URL
mlflow.set_tracking_uri("http://localhost:5000")  # change to your MLflow tracking server address
mlflow.set_experiment("Evidently Drift Metrics")  # change to your experiment name
json_str = report1_metrix
report_dict = json.loads(json_str)

def clean_metric_name(name):
    if "ValueDrift(column=" in name:
        match = re.search(r"ValueDrift\(column=(.*?)\)", name)
        if match:
            return f"drift_{match.group(1)}"
    elif "DriftedColumnsCount" in name:
        return "drifted_column_count"
    cleaned = re.sub(r"[^a-zA-Z0-9_\-./]", "_", name)
    return cleaned[:50]

with mlflow.start_run() as run:
    for metric in report_dict["metrics"]:
        raw_name = metric["metric_id"]
        value = metric["value"]
        if isinstance(value, dict):
            for k, v in value.items():
                metric_name = f"{raw_name}_{k}"
                cleaned_name = clean_metric_name(metric_name)
                mlflow.log_metric(cleaned_name, v)
        else:
            cleaned_name = clean_metric_name(raw_name)
            mlflow.log_metric(cleaned_name, value)

    with open("evidently_report.json", "w") as f:
        json.dump(report_dict, f, indent=2)
    mlflow.log_artifact("evidently_report.json")


🏃 View run capable-crane-119 at: http://localhost:5000/#/experiments/415887779351301810/runs/f0aba4e9d9d541c7b5f9d37fe5e043f9
🧪 View experiment at: http://localhost:5000/#/experiments/415887779351301810
